In [1]:
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [26]:
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();
network = slice.get_networks()[0];
network.config()
subnet = network.get_subnet();
gateway = network.get_gateway();
allocated_ips = network.get_allocated_ips();
available_ips = network.get_available_ips(10);

[IPv4Address('10.145.1.2'), IPv4Address('10.145.1.3'), IPv4Address('10.145.1.4'), IPv4Address('10.145.1.5'), IPv4Address('10.145.1.6'), IPv4Address('10.145.1.7'), IPv4Address('10.145.1.8'), IPv4Address('10.145.1.9'), IPv4Address('10.145.1.10'), IPv4Address('10.145.1.11')] [IPv4Address('10.145.1.1')]


In [24]:
print(available_ips, allocated_ips)

[IPv4Address('10.145.1.2'), IPv4Address('10.145.1.3'), IPv4Address('10.145.1.4'), IPv4Address('10.145.1.5'), IPv4Address('10.145.1.6'), IPv4Address('10.145.1.7'), IPv4Address('10.145.1.8'), IPv4Address('10.145.1.9'), IPv4Address('10.145.1.10'), IPv4Address('10.145.1.11')] [IPv4Address('10.145.1.1')]


In [34]:
def get_ip(node):
    interface = node.get_interface(network_name="NET1")
    return interface.get_ip_addr()

ips = [get_ip(node) for node in nodes];

print(ips);

['10.145.1.2', '10.145.1.3', '10.145.1.4']


In [ ]:
inventory = (
    f"[kube_control_plane]\n"
    f"node1 ansible_host={ips[0]} ip={ips[0]} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
    f"node2 ansible_host=10.145.5.3 ip=10.145.5.3\n"
    f"node3 ansible_host=10.145.5.4 ip=10.145.5.4\n"
)

for i, ip in enumerate(ips):
    inventory + f"node{i + 1} ansoble_host={ip} ip={ip}\n"

In [6]:
async def setup_node(node):
    node.upload_file(local_file_path="node_scripts/node_setup.sh", remote_file_path="setup.sh")
    node.execute(f"chmod +x setup.sh && ./setup.sh");

tasks = [asyncio.create_task(setup_node(node)) for node in nodes]

for task in tasks:
    await task;

Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 Packages [15.0 MB]
Get:5 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe Translation-en [5982 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 Components [3871 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 c-n-f Metadata [301 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 Packages [269 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse Translation-en [118 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 Components [35.0 kB]
Get:12 http://nova.clouds.ar

In [7]:
# nodes[0] is control, rest is worker. Create k8s connections.

print(network.get_subnet(), ips[0]);

nodes[0].upload_file(local_file_path="node_scripts/control_node_start.sh", remote_file_path="start.sh")
nodes[0].execute(f"chmod +x start.sh && ./start.sh {network.get_subnet()} {ips[0]}");

10.145.2.0/24 10.145.2.2
W0404 11:07:05.312663   18446 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W0404 11:07:05.312709   18446 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /var/lib/kubelet /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/super-admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]

The reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d

The reset process does not r

In [ ]:
async def start_node(node):
    print(f"Starting node")
    node.upload_file(local_file_path="node_scripts/worker_node_start.sh", remote_file_path="start.sh")
    node.execute(f"chmod +x start.sh && ./start.sh {ips[0]}");

tasks = [
    asyncio.create_task(start_node(node))
    for node in nodes
]

for task in tasks:
    await task

Starting node 1
10.30.6.60
W0404 08:24:08.749532   28972 preflight.go:56] [reset] WARNING: Changes made to this host by 'kubeadm init' or 'kubeadm join' will be reverted.
[reset] Are you sure you want to proceed? [y/N]: [preflight] Running pre-flight checks
W0404 08:24:08.749619   28972 removeetcdmember.go:106] [reset] No kubeadm config, using etcd pod spec to get data directory
[reset] Deleted contents of the etcd data directory: /var/lib/etcd
[reset] Stopping the kubelet service
[reset] Unmounting mounted directories in "/var/lib/kubelet"
[reset] Deleting contents of directories: [/etc/kubernetes/manifests /var/lib/kubelet /etc/kubernetes/pki]
[reset] Deleting files: [/etc/kubernetes/admin.conf /etc/kubernetes/super-admin.conf /etc/kubernetes/kubelet.conf /etc/kubernetes/bootstrap-kubelet.conf /etc/kubernetes/controller-manager.conf /etc/kubernetes/scheduler.conf]

The reset process does not clean CNI configuration. To do so, you must remove /etc/cni/net.d

The reset process does not